In [8]:
import numpy as np
import pandas as pd
import gurobipy as gp

from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

In [9]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [10]:
file_name = sys.argv[1]

In [11]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = np.zeros(N)
	PP  = np.zeros(N)

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [12]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_mc_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CP = [0]*N
	CR = [0]*N
	KP = 0
	KR = 0 

	for i in range(N):
		CP[i] = PP[i]

		for j in range(i,N):
			CP[i] = CP[i] + HP[j]

	for i in range(N):
		CR[i] = PR[i]
		for j in range(i,N):
			CR[i] += HP[j]
		for j in range(i,N):
			CR[i] -= HR[j]

	for i in range(N):
		AUX = 0 
		for j in range(i+1):
			AUX += D[j]

		KP += HP[i]*AUX

	for i in range(N):
		AUX = 0

		for j in range(i+1):
			AUX += R[j]

		KR += HR[i]*AUX

	K = KR - KP

	try:

		# create a new model
		model = gp.Model("clsr_mc")

		# Create variables
		wp = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wp")
		wr = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wr")
		vor = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="vor")
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")

		# set objective

		fobj = gp.quicksum(CP[i]*xp[i] for i in range(N))
		fobj += gp.quicksum(FP[i]*yp[i] for i in range(N))
		fobj += gp.quicksum(CR[i]*xr[i] for i in range(N))
		fobj += gp.quicksum(FR[i]*yr[i] for i in range(N)) + K
		
		#fobj = 0.0
		#for i in range(N):
		#	fobj += CP[i]*xp[i]

		#for i in range(N):
		#	fobj += yp[i]*FP[i]

		#for i in range(N):
		#	fobj += xr[i]*CR[i]

		#for i in range(N):
		#	fobj += yr[i]*FR[i]

		#fobj += K
		
		model.setObjective(fobj, sense = GRB.MINIMIZE)

		for i in range(N):
			ctr = 0.0
			for j in range(i+1):
				ctr += wp[j,i]
				ctr += wr[j,i]
			model.addConstr(ctr >= D[i])

		for i in range(N):
			ctr = 0.0
			for j in range(i+1):
				ctr+= vor[j,i]
			for j in range(i,N):
				ctr+=(-wr[i,j])
			model.addConstr(ctr == 0)

		for i in range(N):
			ctr =0.0
			for j in range(i,N):
				ctr += vor[i,j]
			model.addConstr(ctr <= R[i])

		for i in range(N):
			for j in range(i,N):
				model.addConstr(wp[i,j] + yp[i]*(-D[j]) <= 0)

		for i in range(N):
			for j in range(i,N):
				model.addConstr(wr[i,j] + yr[i]*(-min(SR[0][i],D[j])) <= 0)

		for i in range(N):
			for j in range(i,N):
				model.addConstr(vor[i,j] + yr[j]*(-R[i]) <= 0)

		for i in range(N):
			ctr = 0.0 
			ctr += xp[i]
			for j in range(i,N):
				ctr += (-wp[i,j])
			model.addConstr(ctr == 0)

		for i in range(N):
			ctr = 0.0
			ctr += xr[i]
			for j in range(i,N):
				ctr += (-wr[i,j])
			model.addConstr(ctr == 0)

		#model.addConstrs(
		#	xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N)
		#	)

		#model.addConstrs(
		#	xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N)
		#	)

		model.addConstrs(
			xp[i] + xr[i] <= C for i in range(N)
			)

		# Parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# turn off display
		gp.setParam('OutputFlag', 0)

		# open log file
		_ = open('clspr_mc.log', 'w')

		# optimize model
		model.optimize()

		tmp=0
		if model.status == GRB.OPTIMAL:
			tmp=1

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	objval = model.ObjVal
	objbound = model.ObjBound
	mipgap = model.MIPGap
	runtime = model.Runtime
	nodecount = model.NodeCount

	return objval, objbound, mipgap, runtime, nodecount, tmp

In [13]:
def main(file_name):

	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, status = clsr_mc_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_mc_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,3))+';'
			   +str(round(status,2))+'\n')

	arquivo.close()

In [14]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,2):
			file_name = f"c{dim}_{id}.txt"			
			main(file_name)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 4446 rows, 4342 columns and 18122 nonzeros
Model fingerprint: 0xdde2805d
Variable types: 4238 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [8e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 129 rows and 76 columns
Presolve time: 0.03s
Presolved: 4317 rows, 4266 columns, 16718 nonzeros
Variable types: 4163 continuous, 103 integer (103 binary)
Found heuristic solution: objective 20800.000000

Root relaxation: objective 8.597783e+0